# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('main_task.csv')

Посмотрим на данные:

In [3]:
df.head(10)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
5,id_1418,Oporto,NaN,1419.0,3.0,NaN,2.0,"[['There are better 3 star hotel bars', 'Amazi...",/Restaurant_Review-g189180-d12503536-Reviews-D...,d12503536
6,id_1720,Milan,"['Italian', 'Pizza']",1722.0,4.0,$,50.0,"[['Excellent simple local eatery.', 'Excellent...",/Restaurant_Review-g187849-d5808504-Reviews-Pi...,d5808504
7,id_825,Bratislava,['Italian'],826.0,3.0,NaN,9.0,"[['Wasting of money', 'excellent cuisine'], ['...",/Restaurant_Review-g274924-d3199765-Reviews-Ri...,d3199765
8,id_2690,Vienna,NaN,2692.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g190454-d12845029-Reviews-G...,d12845029
9,id_4209,Rome,"['Italian', 'Pizza', 'Fast Food']",4210.0,4.0,$,55.0,"[['Clean efficient staff', 'Nice little pizza ...",/Restaurant_Review-g187791-d8020681-Reviews-Qu...,d8020681


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


Отметим наличие 40000 записей и наличие в 3 столбца null-значений.

Отметим, что в столбцах Restaurant_id и Ranking уж очень близко находятся числовые значения. Попробуем немного проверить наши домыслы. Преобразуем эти столбцы к единому виду и проверим их на зависимость.

In [5]:
#заменим строковые значения на просто число в столбце Restaurant_id
df['Restaurant_id'] = df['Restaurant_id'].apply(lambda x: int(x[3:]))
#приведем значения в Ranking к типу int
df['Ranking'] = df['Ranking'].apply(lambda x: int(x))


In [6]:
df.corr(method='pearson')

,Restaurant_id,Ranking,Rating,Number of Reviews
Restaurant_id,1.000000,1.000000,-0.368308,-0.222637
Ranking,1.000000,1.000000,-0.368371,-0.222670
Rating,-0.368308,-0.368371,1.000000,0.030964
Number of Reviews,-0.222637,-0.222670,0.030964,1.000000


Как мы и полагали, данные столбцы линейно зависимы полностью, поэтому в будущем, какой-либо из них мы точно удалим. Однако, глядя на эти данные все же есть вероятность того, что что-то тут не так, т.к. разница на первый взгляд равна 1, но не для всех значений. Поэтому создадим еще один столбец с разнице между Restaurant_id и Ranking.

In [7]:
df['Dif_id_rank'] = df['Ranking'] - df['Restaurant_id']

Посмотрим подробнее на значения в столбце City.

In [8]:
df['City'].unique()

array(['Paris', 'Stockholm', 'London', 'Berlin', 'Munich', 'Oporto',
       'Milan', 'Bratislava', 'Vienna', 'Rome', 'Barcelona', 'Madrid',
       'Dublin', 'Brussels', 'Zurich', 'Warsaw', 'Budapest', 'Copenhagen',
       'Amsterdam', 'Lyon', 'Hamburg', 'Lisbon', 'Prague', 'Oslo',
       'Helsinki', 'Edinburgh', 'Geneva', 'Ljubljana', 'Athens',
       'Luxembourg', 'Krakow'], dtype=object)

Отметим, что скорее всего в городе "Oporto" опечатка, поэтому заменим данное название на нормальное и адекватное "Porto".

In [9]:
df['City'] = df['City'].replace('Oporto', 'Porto')

Посмотрим сколько всего уникальных городов представлено в данных

In [10]:
len(df['City'].unique())

31

Рассмотрим столбец Cuisine Style

In [11]:
df['Cuisine Style'].unique()

array(["['European', 'French', 'International']", nan,
       "['Japanese', 'Sushi', 'Asian', 'Grill', 'Vegetarian Friendly', 'Vegan Options', 'Gluten Free Options']",
       ...,
       "['Steakhouse', 'Barbecue', 'Australian', 'Argentinean', 'South American']",
       "['French', 'American', 'Cafe', 'Healthy', 'Soups']",
       "['French', 'Contemporary', 'Fusion', 'Gastropub']"], dtype=object)

Возьмем из этого столбца информацию о количестве кухонь, представленных в ресторане и создадим новый столбец. Заменив, при этом, все nan значения на 0.

In [12]:
def patt_cuisine_style(cuisine):
    if cuisine == 0:
        return 0
    else:
        pattern_cuisine = re.compile('[ a-zA-Z]+')
        cuisines = pattern_cuisine.findall(cuisine)
        cuisines = [item for item in cuisines if len(item) > 1]
        return len(cuisines)

In [13]:
df['Cuisine Style count'] = df['Cuisine Style'].fillna(0).apply(patt_cuisine_style)

Следующий столбец Rating является нашим целевым показателем, трогать не будем.

Далее столбец Price Range.

In [14]:
df['Price Range'].unique() 

array(['$$ - $$$', nan, '$$$$', '$'], dtype=object)

Имеется 3 ранга цены, и еще nan значения. Заменим все значения на более читабельные категории.

In [15]:
def change_price_range(price_range):
    if price_range == '$':
        return 'Low price'
    if price_range == '$$ - $$$': 
        return 'Medium price'
    if price_range == '$$$$':
        return 'High price'
    else: 
        return 'No price data'
df['Price Range'] = df['Price Range'].apply(change_price_range)

Теперь создадим dummy-переменные на основе данных категорий цен и присоединим их к нашей таблице.

In [16]:
df = df.merge(pd.get_dummies(df['Price Range']), left_index=True, right_index=True)

In [17]:
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,Dif_id_rank,Cuisine Style count,High price,Low price,Medium price,No price data
0,5569,Paris,"['European', 'French', 'International']",5570,3.5,Medium price,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,1,3,0,0,1,0
1,1535,Stockholm,NaN,1537,4.0,No price data,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2,0,0,0,0,1
2,352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353,4.5,High price,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,1,7,1,0,0,0
3,3456,Berlin,NaN,3458,5.0,No price data,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,2,0,0,0,0,1
4,615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621,4.0,Medium price,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,6,3,0,0,1,0


Следующий столбец Number of Reviews в нормальном виде и хранит кол-во отзывов. Имеются nan значения, поэтому заменим их все на 0.

In [18]:
df[['Number of Reviews']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Number of Reviews  37457 non-null  float64
dtypes: float64(1)
memory usage: 312.6 KB


In [19]:
df['Number of Reviews'] = df['Number of Reviews'].fillna(0)

Столбец Reviews содержит 2 отзыва и даты их написания на сайте. Вытянем из этого столбца информацию о количестве отзывов и количестве дат, записав эти данные в новые столбцы

In [20]:
def patt_reviews_text(review):
    pattern_review = re.compile('[ a-zA-Z]+')
    reviews = pattern_review.findall(review)
    reviews = [item for item in reviews if len(item) > 1]
    return reviews

def patt_reviews_dates(review):
    pattern_dates = re.compile('\d\d/\d\d/\d\d\d\d')
    dates = pattern_dates.findall(review)
    return dates

In [21]:
df['Review_text'] = df['Reviews'].apply(patt_reviews_text)
df['Review_dates'] = df['Reviews'].apply(patt_reviews_dates)

Посчитаем еще разницу в днях, между первой и второй датой отзыва. И добавим новый столбец с этими данными.

In [22]:
def calcilate_difference(rev_dates):
    if len(rev_dates) > 1:
        return np.abs((pd.to_datetime(rev_dates[0]) - pd.to_datetime(rev_dates[1])).days)
    else:
        return 0

In [23]:
df['Difference date reviews (days)'] = np.abs(df['Review_dates'].apply(calcilate_difference))

И посчитаем количество дат и текстовых отзывов. И добавим новые столбцы с этими данными.

In [24]:
df['Count text'] = df['Review_text'].apply(lambda x: len(x))
df['Count dates'] = df['Review_dates'].apply(lambda x: len(x))

In [25]:
df.head(5)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,Cuisine Style count,High price,Low price,Medium price,No price data,Review_text,Review_dates,Difference date reviews (days),Count text,Count dates
0,5569,Paris,"['European', 'French', 'International']",5570,3.5,Medium price,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,3,0,0,1,0,"[Good food at your doorstep, A good hotel rest...","[12/31/2017, 11/20/2017]",41,2,2
1,1535,Stockholm,NaN,1537,4.0,No price data,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,0,0,0,0,1,"[Unique cuisine, Delicious Nepalese food]","[07/06/2017, 06/19/2016]",382,2,2
2,352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353,4.5,High price,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,7,1,0,0,0,"[Catch up with friends, Not exceptional]","[01/08/2018, 01/06/2018]",2,2,2
3,3456,Berlin,NaN,3458,5.0,No price data,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,0,0,0,0,1,[],[],0,0,0
4,615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621,4.0,Medium price,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,3,0,0,1,0,"[Best place to try a Bavarian food, Nice build...","[11/18/2017, 02/19/2017]",272,2,2


----------------------------------------

Добавим еще один признак, обозначающий количество символов в идентификаторе ID_TA

In [26]:
df['ID_TA_len'] = df['ID_TA'].apply(lambda x: len(x))

Посмотрим на получившийся Dataframe

In [27]:
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,High price,Low price,Medium price,No price data,Review_text,Review_dates,Difference date reviews (days),Count text,Count dates,ID_TA_len
0,5569,Paris,"['European', 'French', 'International']",5570,3.5,Medium price,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,0,0,1,0,"[Good food at your doorstep, A good hotel rest...","[12/31/2017, 11/20/2017]",41,2,2,8
1,1535,Stockholm,NaN,1537,4.0,No price data,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,0,0,0,1,"[Unique cuisine, Delicious Nepalese food]","[07/06/2017, 06/19/2016]",382,2,2,8
2,352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353,4.5,High price,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,1,0,0,0,"[Catch up with friends, Not exceptional]","[01/08/2018, 01/06/2018]",2,2,2,8
3,3456,Berlin,NaN,3458,5.0,No price data,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,0,0,0,1,[],[],0,0,0,8
4,615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621,4.0,Medium price,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,0,0,1,0,"[Best place to try a Bavarian food, Nice build...","[11/18/2017, 02/19/2017]",272,2,2,8


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [28]:
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,High price,Low price,Medium price,No price data,Review_text,Review_dates,Difference date reviews (days),Count text,Count dates,ID_TA_len
0,5569,Paris,"['European', 'French', 'International']",5570,3.5,Medium price,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,0,0,1,0,"[Good food at your doorstep, A good hotel rest...","[12/31/2017, 11/20/2017]",41,2,2,8
1,1535,Stockholm,NaN,1537,4.0,No price data,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,0,0,0,1,"[Unique cuisine, Delicious Nepalese food]","[07/06/2017, 06/19/2016]",382,2,2,8
2,352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353,4.5,High price,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,1,0,0,0,"[Catch up with friends, Not exceptional]","[01/08/2018, 01/06/2018]",2,2,2,8
3,3456,Berlin,NaN,3458,5.0,No price data,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,0,0,0,1,[],[],0,0,0,8
4,615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621,4.0,Medium price,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,0,0,1,0,"[Best place to try a Bavarian food, Nice build...","[11/18/2017, 02/19/2017]",272,2,2,8


In [29]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Rating', 'Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df['Rating']

In [30]:
X

,Ranking,Number of Reviews,Dif_id_rank,Cuisine Style count,High price,Low price,Medium price,No price data,Difference date reviews (days),Count text,Count dates,ID_TA_len
0,5570,194.0,1,3,0,0,1,0,41,2,2,8
1,1537,10.0,2,0,0,0,0,1,382,2,2,8
2,353,688.0,1,7,1,0,0,0,2,2,2,8
3,3458,3.0,2,0,0,0,0,1,0,0,0,8
4,621,84.0,6,3,0,0,1,0,272,2,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,500,79.0,1,4,0,0,1,0,34,3,2,8
39996,6341,542.0,1,5,0,0,1,0,9,2,2,8
39997,1652,4.0,3,2,0,0,0,1,3127,3,2,7
39998,641,70.0,1,5,0,0,1,0,23,2,2,8


In [31]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [32]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [33]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [34]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [35]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.29622556666666666


---------------------

Получившийся результат существенно лучше предыдущего без каких-либо произведенных действий (0.3814126095238095)

Однако попробуем еще улучшить результат, путем создания новых признаков. Напрашивается вариант с созданием dummy-переменных для городов. Создадим на новом Dataframe.

In [36]:
df_cities = df.merge(pd.get_dummies(df['City']), left_index=True, right_index=True)

In [37]:
df_cities.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,Munich,Oslo,Paris,Porto,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,5569,Paris,"['European', 'French', 'International']",5570,3.5,Medium price,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,0,0,1,0,0,0,0,0,0,0
1,1535,Stockholm,NaN,1537,4.0,No price data,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,0,0,0,0,0,0,1,0,0,0
2,352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353,4.5,High price,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,0,0,0,0,0,0,0,0,0,0
3,3456,Berlin,NaN,3458,5.0,No price data,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,0,0,0,0,0,0,0,0,0,0
4,615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621,4.0,Medium price,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,1,0,0,0,0,0,0,0,0,0


Еще раз обучим модель и получим результат.

In [38]:
X = df_cities.drop(['Rating', 'Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.211146


Результат оказался еще лучше. Попробуем еще поэкспериментировать и что будет, если мы немного поудаляем столбцы в разных комбинациях

In [39]:
X

,Ranking,Number of Reviews,Dif_id_rank,Cuisine Style count,High price,Low price,Medium price,No price data,Difference date reviews (days),Count text,...,Munich,Oslo,Paris,Porto,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,5570,194.0,1,3,0,0,1,0,41,2,...,0,0,1,0,0,0,0,0,0,0
1,1537,10.0,2,0,0,0,0,1,382,2,...,0,0,0,0,0,0,1,0,0,0
2,353,688.0,1,7,1,0,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
3,3458,3.0,2,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,621,84.0,6,3,0,0,1,0,272,2,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,500,79.0,1,4,0,0,1,0,34,3,...,0,0,0,0,0,0,0,0,0,0
39996,6341,542.0,1,5,0,0,1,0,9,2,...,0,0,1,0,0,0,0,0,0,0
39997,1652,4.0,3,2,0,0,0,1,3127,3,...,0,0,0,0,0,0,1,0,0,0
39998,641,70.0,1,5,0,0,1,0,23,2,...,0,0,0,0,0,0,0,0,1,0


## Без количества кухонь

In [40]:
X = df_cities.drop(['Rating', 'Restaurant_id','Cuisine Style count' ,'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2164295


## Без разницы в днях между отзывами

In [41]:
X = df_cities.drop(['Rating', 'Restaurant_id','Difference date reviews (days)' ,'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.211698


## Без количества отзывов

In [42]:
X = df_cities.drop(['Rating', 'Restaurant_id','Number of Reviews' ,'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.27635750000000003


## Без количества отображаемых отзывов

In [43]:
X = df_cities.drop(['Rating', 'Restaurant_id','Count text' ,'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21208400000000002


## Без количества дата отзывов

In [44]:
X = df_cities.drop(['Rating', 'Restaurant_id','Count dates' ,'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21521550000000003


## Без длины ID_TA

In [45]:
X = df_cities.drop(['Rating', 'Restaurant_id','ID_TA_len' ,'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21425950000000002


Как видим, наиболее точный результат все же достиигается когда мы удалим количество кухонь.

Однако, вернемся к исходным данным.

In [46]:
df = pd.read_csv('main_task.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


Имеется большое количество незаполненных значений для 3 столбцов. Попробуем исправить эту ситуацию, путем взятия информации в первоисточнике - сайт TripAdvisor.

Напишем код, который будет актуализировать информацию с сайта по 3 позициям: Cuisine Style, Price Range и Number of Reviews. Отмечу, что актуализироваться будет информация только по ресторанам, которые не имеют данных по Cuisine Style или Number of Reviews.

In [47]:
def fill_actual(row):
    try:
        if row['Cuisine Style'] == 0 or row['Number of Reviews'] == 0:
            page_text = requests.get('https://www.tripadvisor.co.uk'+row['URL_TA']).text
            soup = BeautifulSoup(page_text, 'html.parser')
            cuisines = str(soup.find_all(attrs={"class": "_1XLfiSsv"})[0].text)
            count_reviews = int(str(soup.find_all(attrs={"class": "reviews_header_count"})[0].text)[1:-1])
            price_range = str(soup.find_all(attrs={"class": "_2mn01bsa"})[0].text)
            if '$' not in price_range:
                price_range = 0
            print(row['Restaurant_id'])
            return pd.Series([cuisines, count_reviews, price_range]) 
        else:
            print('Skip '+row['Restaurant_id'])
            return pd.Series([row['Cuisine Style'], row['Number of Reviews'], row['Price Range']])
    except:
        return pd.Series([row['Cuisine Style'], row['Number of Reviews'], row['Price Range']])


Следующий фрагмент кода непосредственно запускает парсинг и записывает актуальные данные в DataFrame. Однако, запускать его я в данном notebook не буду, т.к. процесс формирования всех актуальных данных занял у меня порядка 10 часов. Результаты я сохранил в отдельном файле csv

In [112]:

#df[['Cuisine Style','Number of Reviews', 'Price Range']] = df.apply(fill_actual, axis=1)
#df.to_csv (r'export_dataframe.csv', index = False, header=True)


После сохранения попробуем загрузить и посмотреть на новый DataFrame.

In [48]:
df_new = pd.read_csv('export_dataframe.csv')
df_new.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,"Asian, Nepalese",1537.0,4.0,$$ - $$$,15.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,0.0,3458.0,5.0,0.0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [49]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      40000 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        40000 non-null  object 
 6   Number of Reviews  40000 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


Проведем с данным DataFrame те же операции, что и над исходным.

In [50]:
def patt_cuisine_style(cuisine):
    if cuisine == 0:
        return 0
    else:
        pattern_cuisine = re.compile('[ a-zA-Z]+')
        cuisines = pattern_cuisine.findall(cuisine)
        cuisines = [item for item in cuisines if len(item) > 1]
        return len(cuisines)

def change_price_range(price_range):
    if price_range == '$':
        return 'Low price'
    if price_range == '$$ - $$$': 
        return 'Medium price'
    if price_range == '$$$$':
        return 'High price'
    else: 
        return 'No price data'

def patt_reviews_text(review):
    pattern_review = re.compile('[ a-zA-Z]+')
    reviews = pattern_review.findall(review)
    reviews = [item for item in reviews if len(item) > 1]
    return reviews

def patt_reviews_dates(review):
    pattern_dates = re.compile('\d\d/\d\d/\d\d\d\d')
    dates = pattern_dates.findall(review)
    return dates

def calcilate_difference(rev_dates):
    if len(rev_dates) > 1:
        return np.abs((pd.to_datetime(rev_dates[0]) - pd.to_datetime(rev_dates[1])).days)
    else:
        return 0

df_new['Restaurant_id'] = df_new['Restaurant_id'].apply(lambda x: int(x[3:]))
df_new['Ranking'] = df_new['Ranking'].apply(lambda x: int(x))
df_new['Dif_id_rank'] = df_new['Ranking'] - df_new['Restaurant_id']
df_new['City'] = df_new['City'].replace('Oporto', 'Porto')
df_new['Cuisine Style count'] = df_new['Cuisine Style'].fillna(0).apply(patt_cuisine_style)
df_new['Price Range'] = df_new['Price Range'].apply(change_price_range)
df_new = df_new.merge(pd.get_dummies(df_new['Price Range']), left_index=True, right_index=True)
df_new['Review_text'] = df_new['Reviews'].apply(patt_reviews_text)
df_new['Review_dates'] = df_new['Reviews'].apply(patt_reviews_dates)
df_new['Difference date reviews (days)'] = np.abs(df_new['Review_dates'].apply(calcilate_difference))
df_new['Count text'] = df_new['Review_text'].apply(lambda x: len(x))
df_new['Count dates'] = df_new['Review_dates'].apply(lambda x: len(x))
df_new['ID_TA_len'] = df_new['ID_TA'].apply(lambda x: len(x))

In [51]:
df_new.columns

Index(['Restaurant_id', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA',
       'Dif_id_rank', 'Cuisine Style count', 'High price', 'Low price',
       'Medium price', 'No price data', 'Review_text', 'Review_dates',
       'Difference date reviews (days)', 'Count text', 'Count dates',
       'ID_TA_len'],
      dtype='object')

Теперь обучим новую модель

In [52]:
X = df_new.drop(['Rating', 'Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_new['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.312251


Добавим города, как dummy-переменные

In [53]:
df_new_cities = df_new.merge(pd.get_dummies(df['City']), left_index=True, right_index=True)

In [54]:
X = df_new_cities.drop(['Rating', 'Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_new_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2244505


## Без длины ID

In [55]:
X = df_new_cities.drop(['ID_TA_len','Rating', 'Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_new_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.225557


## Без количества кухонь

In [56]:
X = df_new_cities.drop(['Cuisine Style count','Rating', 'Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA',
            'Review_text', 'Review_dates'], axis = 1)
y = df_new_cities['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.224498


----------------------

# ИТОГ

Таким образом, в результате всех мероприятий получили обученные модели. Наилучший результат был достигнут для неактуализированного набора данных по 3 позициям, и ошибка равна 

MAE: 0.211146
    
Для актуализированных данных наилучшая полученная ошибка равна:

MAE: 0.22346349999999998
    
Такую разницу можно объяснить актуализацией не всех позиций.